In [ ]:
import tkinter as tk
from tkinter import filedialog, ttk, messagebox as tm
from tkinter import Text, END
import spacy
import nltk
from nltk.corpus import wordnet as wn
from spacy.lang.en import English
from gensim.summarization import summarize
from gensim import corpora
import pickle
import gensim
import re
import speech_recognition as sr
import subprocess
import os
import uuid
from punctuator import Punctuator

# Initialize Punctuator
p = Punctuator('Demo-Europarl-EN.pcl')

# Initialize NLP tools
parser = English()
en_stop = set(nltk.corpus.stopwords.words('english'))


class FileProcessor:
    def __init__(self):
        self.datafile = ""
        self.topicfile = ""
        self.summary_ratio = 0.1  # Default summarization ratio
        self.num_keywords = 5  # Default number of keywords
        self.full_text = ""  # Store the full text for re-summarization
        self.tokens = []  # Store tokens for re-extracting topics

    def set_summary_ratio(self, ratio):
        self.summary_ratio = float(ratio)

    def set_num_keywords(self, num):
        self.num_keywords = int(num)

    def tokenize(self, text):
        lda_tokens = []
        tokens = parser(text)
        for token in tokens:
            if not token.orth_.isspace():
                lda_tokens.append(token.lower_)
        return lda_tokens

    def get_lemma(self, word):
        lemma = wn.morphy(word)
        return lemma if lemma else word

    def prepare_text_for_lda(self, text):
        tokens = self.tokenize(text)
        tokens = [token for token in tokens if len(token) > 4 and token not in en_stop]
        tokens = [self.get_lemma(token) for token in tokens]
        return tokens

    def summarize_text(self, text):
        if len(text) < 1000:
            raise ValueError("Text too short for summarization")
        return summarize(text, ratio=self.summary_ratio)

    def extract_topics(self, text):
        self.tokens = self.prepare_text_for_lda(text)  # Store tokens for re-extraction
        dictionary = corpora.Dictionary([self.tokens])
        corpus = [dictionary.doc2bow(self.tokens)]
        ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)
        topics = ldamodel.print_topics(num_words=self.num_keywords)  # Use self.num_keywords
        return self.format_topics(topics)

    def format_topics(self, topics):
        test_string = topics[0][1]
        res = re.findall(r'\w+', test_string)
        res = [''.join(x for x in i if x.isalpha()) for i in res]
        return ' '.join(res)

    def process_text_file(self, filename):
        try:
            with open(filename, encoding="utf8") as f:
                self.full_text = ' '.join(line.strip() for line in f if line.strip())
            return self.full_text
        except Exception as e:
            raise ValueError(f"Error reading file: {e}")

    def process_audio_file(self, filename):
        try:
            filename1 = "audio.wav"
            command = [r"D:\ffmpeg-master-latest-win64-gpl\bin\ffmpeg.exe", "-y", "-i", filename, filename1]
            subprocess.run(command, shell=True, capture_output=True)
            r = sr.Recognizer()
            with sr.AudioFile(filename1) as source:
                r.adjust_for_ambient_noise(source)
                audio = r.record(source)
            self.full_text = p.punctuate(r.recognize_sphinx(audio))
            return self.full_text
        except Exception as e:
            raise ValueError(f"Error processing audio: {e}")


class Application(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("ARTICLE SUMMARISER")
        self.state('zoomed')
        self.file_processor = FileProcessor()
        self.create_widgets()

    def create_widgets(self):
        # Buttons
        self.button1 = tk.Button(self, text='Upload Text File', command=self.upload_text)
        self.button1.grid(row=0, column=0, padx=10, pady=10)

        self.button4 = tk.Button(self, text='Upload\nAudio/Video\nFile', command=self.upload_audio)
        self.button4.grid(row=1, column=0, padx=10, pady=10)

        self.button2 = tk.Button(self, text='Topics', command=self.display_topics)
        self.button2.grid(row=2, column=0, padx=10, pady=10)

        self.button3 = tk.Button(self, text='Summarize', command=self.display_summary)
        self.button3.grid(row=3, column=0, padx=10, pady=10)

        # Summary Ratio Slider
        self.ratio_label = tk.Label(self, text="Summarization Ratio:")
        self.ratio_label.grid(row=4, column=0, padx=10, pady=10)
        self.ratio_slider = ttk.Scale(self, from_=0.05, to=0.5, command=self.set_summary_ratio)
        self.ratio_slider.set(0.1)
        self.ratio_slider.grid(row=5, column=0, padx=10, pady=10)

        # Number of Keywords Entry
        self.keywords_label = tk.Label(self, text="Number of Keywords:")
        self.keywords_label.grid(row=6, column=0, padx=10, pady=10)
        self.keywords_entry = tk.Entry(self)
        self.keywords_entry.insert(0, "5")
        self.keywords_entry.grid(row=7, column=0, padx=10, pady=10)

        # Text Boxes
        self.textBox2 = Text(self, height=8, width=100)
        self.textBox2.grid(row=0, column=1, rowspan=4, padx=10, pady=10)

        self.textBox1 = Text(self, height=13, width=100)
        self.textBox1.grid(row=5, column=1, rowspan=4, padx=10, pady=10)

        # Labels
        self.keywords_label = tk.Label(self, text="Relevant Keywords")
        self.keywords_label.grid(row=0, column=2, padx=10, pady=10)

        self.summary_label = tk.Label(self, text="Summary of the topic")
        self.summary_label.grid(row=5, column=2, padx=10, pady=10)

    def set_summary_ratio(self, value):
        self.file_processor.set_summary_ratio(value)

    def set_num_keywords(self):
        try:
            num = self.keywords_entry.get()
            self.file_processor.set_num_keywords(num)
        except ValueError:
            tm.showerror("Error", "Invalid number of keywords")

    def upload_text(self):
        filename = filedialog.askopenfilename()
        if not filename:
            return
        try:
            full_text = self.file_processor.process_text_file(filename)
            summary = self.file_processor.summarize_text(full_text)
            topics = self.file_processor.extract_topics(full_text)
            self.file_processor.datafile = [summary]
            self.file_processor.topicfile = topics
            tm.showinfo("Success", "File processed successfully!")
        except Exception as e:
            tm.showerror("Error", str(e))

    def upload_audio(self):
        filename = filedialog.askopenfilename()
        if not filename:
            return
        try:
            full_text = self.file_processor.process_audio_file(filename)
            summary = self.file_processor.summarize_text(full_text)
            topics = self.file_processor.extract_topics(full_text)
            self.file_processor.datafile = [summary]
            self.file_processor.topicfile = topics
            tm.showinfo("Success", "Audio processed successfully!")
        except Exception as e:
            tm.showerror("Error", str(e))

    def display_topics(self):
        self.set_num_keywords()  # Update number of keywords
        try:
            # Re-extract topics with the updated number of keywords
            topics = self.file_processor.extract_topics(self.file_processor.full_text)
            self.file_processor.topicfile = topics
            self.textBox2.delete('1.0', END)
            self.textBox2.insert(tk.END, self.file_processor.topicfile)
        except Exception as e:
            tm.showerror("Error", str(e))

    def display_summary(self):
        try:
            # Re-summarize with the updated ratio
            summary = self.file_processor.summarize_text(self.file_processor.full_text)
            self.file_processor.datafile = [summary]
            self.textBox1.delete('1.0', END)
            self.textBox1.insert(tk.END, self.file_processor.datafile[0])
        except Exception as e:
            tm.showerror("Error", str(e))


if __name__ == "__main__":
    app = Application()
    app.mainloop()

Selected: C:/Users/A/Machine-Learning-master/reviewss.txt
Hello <_io.TextIOWrapper name='C:/Users/A/Machine-Learning-master/reviewss.txt' mode='r' encoding='utf8'>
[(0, '0.005*"citizenship" + 0.005*"student" + 0.005*"amendment" + 0.005*"india" + 0.005*"protest"'), (1, '0.041*"citizenship" + 0.023*"india" + 0.021*"protest" + 0.021*"amendment" + 0.021*"student"')]
  citizenship   student   amendment   india   protest 
A person who stays beyond the visa permit is also referred to as an illegal immigrant.When did the issue of Citizenship (Amendment) Bill come up?Prior to the 2014 Lok Sabha elections, the Bharatiya Janata Party (BJP), which was seeking to topple the Congress-led UPA government, promised to grant citizenship to Hindus persecuted in the neighbouring countries.
  citizenship   student   amendment   india   protest 
['A person who stays beyond the visa permit is also referred to as an illegal immigrant.When did the issue of Citizenship (Amendment) Bill come up?Prior to the 2014